# Raw code

In [ ]:
%%writefile raw_add.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}


int main() {
  int res=0;
  int *d_res = NULL;

  // Launch add() kernel on GPU
  add<<<1,1>>>(2, 2, d_res);

  cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost);
  printf("2 + 2 = %d\n", res);

  return EXIT_SUCCESS;
}

Writing raw_add.cu


In [ ]:
!nvcc -arch=sm_75 raw_add.cu -o raw_add

In [ ]:
!./raw_add

2 + 2 = 0


# Debugging

In [ ]:
%%writefile add.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}
int main() {
  int res=0;
  int *d_res = NULL;

  // Launch add() kernel on GPU
  add<<<1,1>>>(2, 2, d_res);

  cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost);
  printf("2 + 2 = %d\n", res);

  return EXIT_SUCCESS;
}

Overwriting add.cu


In [ ]:
! nvcc -arch=sm_75 -g -G add.cu -o add

In [ ]:
%%writefile debug_instructions.txt

set cuda api_failures stop
catch throw
r
bt
info locals
thread 1
bt

Overwriting debug_instructions.txt


In [ ]:
! cuda-gdb -batch -x debug_instructions.txt ./add

Catchpoint 1 (throw)
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
[New Thread 0x7ffff5fff000 (LWP 11870)]
[New Thread 0x7ffff4bff000 (LWP 11871)]
[Detaching after fork from child process 11872]
[New Thread 0x7fffeefde000 (LWP 11877)]

CUDA Exception: Warp Illegal Address
The exception was triggered at PC 0x7fffc903fc30  add(int, int, int*)  (add.cu:5)

Thread 1 "add" received signal CUDA_EXCEPTION_14, Warp Illegal Address.
[Switching focus to CUDA kernel 0, grid 1, block (0,0,0), thread (0,0,0), device 0, sm 0, warp 0, lane 0]
0x00007fffc903fc50 in add<<<(1,1,1),(1,1,1)>>> (a=2, b=2, res=0x0) at add.cu:6
6	}
#0  0x00007fffc903fc50 in add<<<(1,1,1),(1,1,1)>>> (a=2, b=2, res=0x0) at add.cu:6
No locals.
[Switching to thread 1 (Thread 0x7ffff7d79000 (LWP 11866))]
#0  0x00007ffff61ad970 in cudbgReportDriverApiError () from /usr/lib64-nvidia/libcuda.so.1
#0  0x00007ffff61ad970 in cudbgReportDriverApiError () from /us

# Code with error management

In [ ]:
%%writefile add.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}


int main() {
  int res=0;
  int *d_res = NULL;
  cudaError_t err;

  // Launch add() kernel on GPU
  add<<<1,1>>>(2, 2, d_res);
  err = cudaPeekAtLastError();
  if (err != cudaSuccess){
      fprintf(stderr,"GPUassert: add launch failed with the error : %s \n", cudaGetErrorString(err));
      exit(err);
   }
  err = cudaDeviceSynchronize() ;
  if (err != cudaSuccess){
      fprintf(stderr,"GPUassert: add execution failed with the error : %s \n", cudaGetErrorString(err));
      exit(err);
  }

  err = cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost);
  if (err != cudaSuccess){
      fprintf(stderr,"GPUassert: cudaMemcpy failed with the error : %s \n", cudaGetErrorString(err));
      exit(err);
   }

  printf("2 + 2 = %d\n", res);

  return EXIT_SUCCESS;
}

Overwriting add.cu


In [ ]:
! nvcc -arch=sm_75 add.cu -o add

In [ ]:
!./add

GPUassert: add execution failed with the error : an illegal memory access was encountered 


# CUDA error management Utilities in a separate cell


In [ ]:
%%writefile cuda_stuff.cuh
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#ifndef cuda_stuff_H
#define cuda_stuff_H

//MACRO TO DEBUG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#endif


Overwriting cuda_stuff.cuh


In [ ]:
%%writefile addition.cu
#include <stdio.h>
#include <stdlib.h>

#include "cuda_stuff.cuh"

__global__ void add(int a, int b, int *res) {
  *res = a + b;
}


int main() {
  int res=0;
  int *d_res = NULL;

  // Launch add() kernel on GPU
  add<<<1,1>>>(2, 2, d_res);
  gpuErrchk( cudaPeekAtLastError() );
  gpuErrchk( cudaDeviceSynchronize() );

  gpuErrchk(cudaMemcpy(&res, d_res, sizeof(int), cudaMemcpyDeviceToHost));
  printf("2 + 2 = %d\n", res);

  return EXIT_SUCCESS;
}

Overwriting addition.cu


In [ ]:
!nvcc -arch=sm_75 addition.cu -o addition

In [ ]:
! ./addition

GPUassert: an illegal memory access was encountered addition.cu 18


>> Thanks to the Debbuging, we now know that we are trying to access an illegal memory adresse, we deduce naturally that we forgot to allocate memory with **cudaMalloc()**.